In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
from astropy.cosmology import Planck18 as cosmo
from lbg_desc_forecast import get_lbg_mappers, fig_dir

In [ ]:
def nz(z):
    z0 = 0.9 / np.sqrt(2)
    n = 1 / (2 * z0) * (z / z0) ** 2 * np.exp(-((z / z0) ** (3 / 2)))
    n /= np.sum(n)
    return n

In [ ]:
mappers = get_lbg_mappers(10)
for mapper in mappers:
    mapper.f_interlopers = 0

In [ ]:
fig, ax = plt.subplots(figsize=(5, 2), dpi=150, constrained_layout=True)

# LSST low-z tomographic bins
# ---------------------------

# calculate n(z) on a fine grid
zfine = np.linspace(0, 4, 1000)
nzfine = nz(zfine)

# separate redshift range into bins with equal numbers of galaxies
nbins = 10
bin_idx = np.abs(
    nzfine.cumsum()[None, :] - np.linspace(0, 1, nbins + 1)[:, None]
).argmin(axis=1)

# loop over the source
for i in range(nbins):
    # get the true n(z) for the bin
    nzbin = nzfine.copy()
    nzbin[: bin_idx[i]] = nzbin[bin_idx[i + 1] :] = 0

    # calculate the photo-z error for the bin
    zmean = zfine[nzbin > 0].mean()
    sigma = 0.03 * (1 + zmean) / (zfine[1] - zfine[0])

    # convolve the true distribution with the Gaussian error
    nzbin = gaussian_filter(nzbin, sigma)

    # normalize n(z) for the bin
    nzbin /= np.trapz(nzbin, zfine)

    # plot the distribution
    ax.plot(zfine, nzbin, c="k", lw=0.5)


# Lyman-break galaxies
# --------------------
z, pz = mappers[0].tomographic_bin.pz
ax.plot(z, 1.75 * pz, c="C4")

z, pz = mappers[1].tomographic_bin.pz
ax.plot(z, 1.75 * pz, c="C0")

z, pz = mappers[2].tomographic_bin.pz
ax.plot(z, 1.75 * pz, c="C2")


# CMB Lensing kernel
# ------------------
z = np.linspace(0, 6, 1000)
wk = (
    (1 + z)
    / cosmo.H(z)
    * cosmo.comoving_distance(z)
    * (1 - cosmo.comoving_distance(z) / cosmo.comoving_distance(1100))
)
ax.plot(z, 2.3 * wk / wk.max(), c="silver", zorder=0, lw=1, alpha=0.9)
ax.text(5.25, 1.9, "$W_\kappa$", color="dimgray", fontsize=11)


ax.text(1.2, 5.6, "low-redshift\n3x2pt cosmology", ha="left")
ax.text(4, 4.35, "high-redshift cosmology\nwith 175 million LBGs", ha="center")

ax.set(
    xlim=(0, 5.6),
    ylim=(0, None),
    xlabel="Redshift",
    ylabel="$n(z)$",
)

fig.savefig(fig_dir / "redshift_distributions.pdf", bbox_inches="tight")